<a href="https://colab.research.google.com/github/jhodgens17/audio_classification/blob/main/MVP_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import packages
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
target_words = ['down', 'go', 'left', 'no', 'off',
           'on', 'right', 'stop', 'up', 'yes']

In [ ]:
unknown = ['bed', 'bird', 'cat', 'dog',  
           'eight', 'five', 'four', 'happy', 
           'house', 'marvin', 'nine', 'one', 'seven',
           'sheila', 'six', 'three', 'tree', 'two',
           'wow', 'zero']

In [ ]:
all_words = target_words + unknown

In [ ]:
len(all_words)

30

In [ ]:
all_files = {}
for word in all_words:
  path = '/content/drive/MyDrive/train/audio/'+word
  file_list = os.listdir(path)
  all_files[word] = file_list

In [ ]:
len(all_files)

30

In [ ]:
for word in all_files:
  print(len(all_files[word]))

2359
2372
2353
2375
2357
2367
2367
2380
2375
2377
1713
1731
1733
1746
2352
2357
2372
1742
1750
1746
2364
2370
2377
1734
2369
2356
1733
2373
1745
2376


In [ ]:
total = 0
for word in all_files:
  total = total + len(all_files[word])

In [ ]:
total

64721

In [ ]:
all_paths = {}
for word in all_files:
  paths = []
  for file in all_files[word]:
    path = '/content/drive/MyDrive/train/audio/' + word + '/' + file
    paths.append(path)
  all_paths[word] = paths

In [ ]:
len(all_paths)

30

In [ ]:
total_paths = 0
for word in all_paths:
  total_paths = total_paths + len(all_paths[word])

In [ ]:
total_paths

64721

In [ ]:
y = []
all_paths = []
for target in all_files:
  if target in target_words:
    for file in all_files[target]:
      path = '/content/drive/MyDrive/train/audio/' + target + '/' + file
      all_paths.append(path)
      y.append(target)
  else:
    for file in all_files[target]:
      path = '/content/drive/MyDrive/train/audio/' + target + '/' + file
      all_paths.append(path)
      y.append('unknown')

In [ ]:
len(y)

64721

In [ ]:
len(all_paths)

64721

In [ ]:
y[50000]

'unknown'

In [ ]:
all_paths[50000]

'/content/drive/MyDrive/train/audio/seven/264f471d_nohash_3.wav'

In [ ]:
#Getting all of the files
files_ds = tf.data.Dataset.from_tensor_slices(all_paths)

In [ ]:
#Defining function to get waveform for each clip
def get_waveform(path):
  audio_binary = tf.io.read_file(path)
  audio, sample_rate = tf.audio.decode_wav(audio_binary)
  waveform = tf.squeeze(audio, axis=-1)
  return waveform

In [ ]:
#Getting the waveform for each file
waveform_ds = files_ds.map(get_waveform)

In [ ]:
#Defining a function that makes all audio clips same length and gets spectrogram
def get_spectrogram(waveform):
  zero_padding = tf.zeros([16000] - tf.shape(waveform), dtype=tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(equal_length, frame_length=255, frame_step=128)
  spectrogram = tf.abs(spectrogram)
  return spectrogram

In [ ]:
#Getting the spectrogram for each file
spectrogram_ds = waveform_ds.map(get_spectrogram)

In [ ]:
spectrogram_ds

<MapDataset shapes: (None, 129), types: tf.float32>

In [ ]:
X = []
for spec in spectrogram_ds:
  X.append(spec.numpy())

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

(64721, 124, 129)

In [ ]:
y.shape

(64721,)

In [ ]:
y_d = pd.get_dummies(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_d)

In [ ]:
model = Sequential()

In [ ]:
for spectrogram in spectrogram_ds.take(1):
  input_shape = spectrogram.shape
print('Input shape:', input_shape)

Input shape: (124, 129)


In [ ]:
num_labels = 11

In [ ]:
model.add(Input(shape=input_shape))
model.add(Flatten())
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(num_labels))

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
759/759 [==============================] - 4s 3ms/step - loss: 0.3919 - accuracy: 0.6284
Epoch 2/50
759/759 [==============================] - 2s 3ms/step - loss: 0.3526 - accuracy: 0.6334
Epoch 3/50
759/759 [==============================] - 3s 3ms/step - loss: 0.3571 - accuracy: 0.6347
Epoch 4/50
759/759 [==============================] - 2s 3ms/step - loss: 0.3541 - accuracy: 0.6374
Epoch 5/50
759/759 [==============================] - 2s 3ms/step - loss: 0.3692 - accuracy: 0.6401
Epoch 6/50
759/759 [==============================] - 2s 3ms/step - loss: 0.3686 - accuracy: 0.6413
Epoch 7/50
759/759 [==============================] - 2s 3ms/step - loss: 0.3697 - accuracy: 0.6413
Epoch 8/50
759/759 [==============================] - 3s 3ms/step - loss: 0.3639 - accuracy: 0.6435
Epoch 9/50
759/759 [==============================] - 3s 3ms/step - loss: 0.3601 - accuracy: 0.6463
Epoch 10/50
759/759 [==============================] - 2s 3ms/step - loss: 0.3645 - accuracy: 0.6484

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis = 1)

In [ ]:
y_test_arr = y_test.idxmax(axis = 1)

In [ ]:
y_test.columns

Index(['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'unknown',
       'up', 'yes'],
      dtype='object')

In [ ]:
tar_dict = {0:'down', 1:'go', 2:'left', 3:'no', 4:'off',
            5:'on', 6:'right', 7:'stop', 8:'unknown', 9:'up', 10: 'yes'}

In [ ]:
y_hat = []
for value in y_pred:
  y_hat.append(tar_dict[value])

In [ ]:
y_hat_arr = np.array(y_hat)

In [ ]:
accuracy_score(y_test_arr, y_hat_arr)

0.6540386873493603

In [ ]:
#Get test data for kaggle scoring
!p7zip -d '/content/drive/MyDrive/test.7z'

/usr/bin/p7zip: cannot read /content/drive/MyDrive/test.7z


In [ ]:
model.save('/content/drive/MyDrive/model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/model/assets


In [ ]:
#Getting all of the test files
test_files_ds = tf.data.Dataset.from_tensor_slices(test_paths)

In [ ]:
#Getting the waveform for each test file
test_waveform_ds = test_files_ds.map(get_waveform)

In [ ]:
#Getting the spectrogram for each file
test_spectrogram_ds = test_waveform_ds.map(get_spectrogram)

In [ ]:
X_k = []
for spec in test_spectrogram_ds:
  X_k.append(spec.numpy())

KeyboardInterrupt: ignored

In [ ]:
model2 = Sequential()

In [ ]:
model2.add(Input(shape=input_shape))
model2.add(Flatten())
model2.add(Dense(12, activation='relu'))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(num_labels))

In [ ]:
model2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
infile = open("model.pickle",'rb')
model3 = pickle.load(infile)
infile.close()

EOFError: ignored

In [ ]:
X.shape

(64721, 124, 129)

In [ ]:
X_res = X.reshape(X.shape[0], -1)

In [ ]:
np.savetxt('train_spec.txt', X_res)

In [ ]:
np.savetxt('train_labels.txt', y)

TypeError: ignored